In [1]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
import scipy as sp
sns.set_context('paper')
np.set_printoptions(suppress=True)
# pd.set_option('display.max_columns', None)

In [2]:
def simu_success(tag, params):
    # which task
    path = "simuS1_data/simuS1_design.pkl"
    if tag == "Asso-CR":
        # path = "../Data/simuS1_group1_design.pkl"
        nitems = 48*4  # 96
        test1_num = 40
        i = 1
    elif tag == "Pair-CR":
        # path = "../Data/simuS1_group2_design.pkl"
        nitems = 48*4  # 176
        test1_num = 80
        i = 2
    elif tag == "Item-CR":
        # path = "../Data/simuS1_group3_design.pkl"
        nitems = 48*4  # 136
        test1_num = 80
        i = 3

    # load stimuli
    with open(path, 'rb') as inp:
        df_study = pickle.load(inp)
        df_test = pickle.load(inp)
    df_study = df_study.query(f"group == {i}")
    df_test = df_test.query(f"group == {i}")

    # load semantic matrix
    s_mat = np.load('../../Data/wordpools/ltp_FR_similarity_matrix.npy')

    # update n_item in params to fit the study
    params.update(nitems_in_accumulator = nitems)
    # print(params)

    # run CMR
    df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat)
    df_simu['test'] = df_test['test']
    df_simu = df_simu.merge(df_test,on=['session','list','test','test_itemno1','test_itemno2'])

    # get f_in
    sessions = np.unique(df_simu.session)
    tmp_corr_fin = []
    tmp_omax_fin = []

    for sess in sessions:
        df_tmp = df_study.loc[df_study.session == sess]
        tmp1 = df_tmp.study_itemno1.to_numpy()
        tmp2 = df_tmp.study_itemno2.to_numpy()
        df_tmp2 = df_test.loc[df_test.session == sess]
        tmp3 = df_tmp2.test_itemno1[df_tmp2.test_itemno1 >= 0].to_numpy()
        tmp4 = df_tmp2.test_itemno2[df_tmp2.test_itemno2 >= 0].to_numpy()
        tmp = np.concatenate((tmp1, tmp2, tmp3, tmp4))
        tmp = np.unique(tmp)  # sort
        nlists = len(np.unique(df_simu.list))

        for lst in range(nlists):
            tmp_corr = df_simu.query(f"session == {sess} and list == {lst}")["correct_ans"][test1_num:]
            corrid = np.searchsorted(tmp, tmp_corr)
            corr_fin = [f_in_dif[sess][lst*int(test1_num/2)+i][id] for i, id in enumerate(corrid)]
            omax_fin = [np.max(np.delete(f_in_dif[sess][lst*int(test1_num/2)+i], id)) for i, id in enumerate(corrid)]

            tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
            tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

    df_simu['corr_fin'] = tmp_corr_fin
    df_simu['omax_fin'] = tmp_omax_fin

    # optimal threshold for test1 (only when manually!!)
    # csim_two = df_simu.query("test==1").groupby("correct_ans").csim.mean()
    # opt_thresh = np.mean(csim_two)
    # df_simu['s_resp'] = df_simu.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

    return df_simu

In [3]:
# anal_perform used by pso
import sys
sys.path.append("../../Modeling/CMR_IA/fitting")
from object_funcs import anal_perform_S1 as anal_perform

In [4]:
# def anal_perform(df_simu):

#     # get correctness
#     df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans

#     # recognition performance
#     df_recog = df_simu.query("test==1")
#     hr_far = df_recog.groupby("correct_ans")["s_resp"].mean().to_frame(name="Yes rate")
#     hr = hr_far['Yes rate'][1]
#     far = hr_far['Yes rate'][0]
#     z_hr_far = sp.stats.norm.ppf(hr_far)
#     d_prime = z_hr_far[1].item() - z_hr_far[0].item()
#     print("recognition: \n", hr_far)
#     print("d_prime: ", d_prime)

#     # cued recall performance
#     df_cr = df_simu.query("test==2")
#     p_rc = df_cr.correct.mean()
#     print("cued recall: \n", p_rc)

#     # analyze pair
#     def get_pair(df_tmp):
#         df_tmp_pair = pd.pivot_table(df_tmp,index="pair_idx",columns="test",values="correct")
#         df_tmp_pair.columns = ["test1","test2"]
#         df_tmp_pair.reset_index(inplace=True)
#         return df_tmp_pair

#     df_simu_p = df_simu.query("pair_idx >= 0")
#     df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
#     test2_rsp = pd.Categorical(df_pair.test2, categories=[0,1])
#     test1_rsp = pd.Categorical(df_pair.test1, categories=[0,1])
#     df_tab = pd.crosstab(index=test2_rsp,columns=test1_rsp, rownames=['test2'], colnames=['test1'], normalize=False, dropna=False)
#     print("contingency table: \n", df_tab)
#     # print("contingency table norm: \n", pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize='all'))

#     # compute" Q
#     def Yule_Q(A, B, C, D):
#         return (A * D - B * C) / (A * D + B * C)
#     q = Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)  # add 0.5
#     print("Q: ", q)

#     return p_rc, hr, far, q

### Study Params

In [5]:
params = cmr.make_default_params()
# learn while retrieving
# multiple lists, so beta_rec_post
# correct session num - done
# it seems that the size of all potential words matters, the more, the worse CR. RISK IT!

In [6]:
# # manually adjusted params
# # study params should be the same across three groups
# params.update(
#     beta_enc = 0.3,
#     gamma_fc = 0.2,
#     gamma_cf = 0.2,
#     s_fc = 0.2,
#     s_cf = 0.2,
#     phi_s = 1,
#     phi_d = 0.6,
#     d_ass = 1,
#     learn_while_retrieving = False,
#     var_enc = 1,
#     bad_enc_ratio = 1,
# )
# params.update(
#     beta_cue = 0.4,
#     beta_rec = 0.1,  # beta for retrieved item
#     beta_rec_post = 1,
#     beta_distract = 0.1,
#     # beta_rec_post = 0.1,
#     c_thresh_itm = 0.01,
#     c_thresh_ass = 0.01,
#     c_thresh = 0.01,
#     kappa = 0.01,
#     lamb = 0.002,
#     eta = 0.002,
#     omega = 3,
#     alpha = 1
# )
# params

In [7]:
# pso results
import sys
sys.path.append("../../Modeling/CMR_IA/fitting")
from optimization_utils import make_boundary

_,_,what_to_fit = make_boundary(sim_name='S1')
optim_params = np.loadtxt("../../Modeling/CMR_IA/fitting/outparams_backup/S1_230810_100-200.txt")
for pname, pvalue in zip(what_to_fit, optim_params):
    params[pname] = pvalue
# params.update(beta_rec_post=1)
params.update(learn_while_retrieving=True, rec_time_limit=10000)
params

{'beta_enc': 0.417666,
 'beta_rec': 0.651769,
 'beta_cue': 0.847962,
 'beta_rec_post': 0.584781,
 'beta_distract': 0.458767,
 'phi_s': 4.102046,
 'phi_d': 0.028525,
 's_cf': 0.475552,
 's_fc': 0.33572,
 'kappa': 0.12235,
 'eta': 0.080522,
 'omega': 6.365086,
 'alpha': 0.858893,
 'c_thresh': 0.309278,
 'c_thresh_itm': 0.498363,
 'c_thresh_ass': 0.62317,
 'd_ass': 0.90193,
 'lamb': 0.194837,
 'rec_time_limit': 10000,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'thresh_sigma': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.209602,
 'gamma_cf': 0.351656}

### Association - CR (Group1)

In [8]:
tag = "Asso-CR"
df_simu_g1 = simu_success(tag, params)
df_simu_g1

CMR2 Time: 112.3355324268341


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,subject,group,corr_fin,omax_fin
0,0,0,1213,926,1.0,51.305733,0.698342,1,ROUGE,MOUTH,0,1,-1,0,1,-1.000000,-1.000000
1,0,0,830,717,1.0,180.229340,0.635520,1,LIST,HORIZON,1,1,34,0,1,-1.000000,-1.000000
2,0,0,577,1183,0.0,837.369629,0.558718,1,FLEA,RELISH,1,1,0,0,1,-1.000000,-1.000000
3,0,0,588,439,1.0,59.694347,0.690770,1,FOLDER,DANDRUFF,1,0,25,0,1,-1.000000,-1.000000
4,0,0,687,1311,1.0,53.512703,0.696236,1,HANGER,SIRLOIN,1,1,22,0,1,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,99,4,1169,-1,1151.0,560.000000,0.758642,2,RAT,-1,331,0,226,99,1,0.524507,8.493725
29996,99,4,1349,-1,1151.0,540.000000,0.773112,2,SPHINX,-1,848,1,219,99,1,1.527869,8.961842
29997,99,4,1307,-1,1151.0,510.000000,0.791532,2,SILK,-1,72,0,194,99,1,2.464262,9.527805
29998,99,4,984,-1,1151.0,550.000000,0.739035,2,ORGAN,-1,899,1,196,99,1,2.496073,8.731302


In [9]:
# df_simu_g1.query("test==2 and session == 0 and list == 0")

In [10]:
subjects = np.unique(df_simu_g1.subject)
g1_stats = []
for subj in subjects:
    df_subj = df_simu_g1.query(f"subject == {subj}").copy()
    # df_subj = df_simu_g1.query(f"subject == {subj} and list == 0").copy()
    g1_stats.append(list(anal_perform(df_subj)))

In [11]:
print(np.array(g1_stats))
print(np.mean(np.array(g1_stats),axis=0))
print(sp.stats.sem(np.array(g1_stats),axis=0))

[[0.42       0.82       0.13       0.72562944]
 [0.43       0.71       0.18       0.97803468]
 [0.4        0.77       0.22       0.87797147]
 [0.36       0.7        0.15       0.66505558]
 [0.43       0.71       0.12       0.87474333]
 [0.37       0.77       0.15       0.53075571]
 [0.42       0.75       0.14       0.90313524]
 [0.26       0.77       0.13       0.92058597]
 [0.41       0.83       0.11       0.82208589]
 [0.33       0.76       0.17       0.83925346]
 [0.41       0.84       0.14       0.80777538]
 [0.45       0.8        0.19       0.79131886]
 [0.41       0.72       0.05       0.85183547]
 [0.45       0.82       0.16       0.7581139 ]
 [0.39       0.82       0.14       0.55319149]
 [0.37       0.83       0.14       0.93156733]
 [0.29       0.74       0.1        0.94344313]
 [0.47       0.78       0.11       0.90701001]
 [0.4        0.84       0.18       0.7994122 ]
 [0.5        0.75       0.11       0.82926829]
 [0.24       0.83       0.21       0.62068966]
 [0.31       

In [24]:
g1_stats = np.array(g1_stats)
g1_ds = sp.stats.norm.ppf(np.array(g1_stats[:,1])) - sp.stats.norm.ppf(np.array(g1_stats[:,2]))
print(np.mean(g1_ds))
print(sp.stats.sem(g1_ds))

1.867779982785053
0.021002321352824628


### Pair - CR (Group2)

In [12]:
tag = "Pair-CR"
df_simu_g2 = simu_success(tag, params)
df_simu_g2

CMR2 Time: 108.51171255111694


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,subject,group,corr_fin,omax_fin
0,0,0,192,1402,1.0,190.916809,0.632640,1,BULB,SUIT,1,1,13,0,2,-1.000000,-1.000000
1,0,0,810,1481,0.0,539.798706,0.580672,1,LEMON,TOOTH,0,-1,-1,0,2,-1.000000,-1.000000
2,0,0,384,686,1.0,14.452640,0.761688,1,COTTAGE,HANDKERCHIEF,1,1,8,0,2,-1.000000,-1.000000
3,0,0,1317,1137,0.0,9.500948,0.214064,1,SKILLET,PUBLISHER,0,-1,-1,0,2,-1.000000,-1.000000
4,0,0,506,977,0.0,2410.055908,0.505861,1,DUNE,ONION,0,-1,-1,0,2,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35995,99,2,120,-1,918.0,590.000000,0.739653,2,BELT,-1,917,0,97,99,2,2.366999,7.336515
35996,99,2,328,-1,918.0,650.000000,0.696525,2,CLOUD,-1,869,0,113,99,2,1.018910,6.631553
35997,99,2,456,-1,918.0,600.000000,0.727964,2,DIAGRAM,-1,1582,0,110,99,2,2.203162,7.545782
35998,99,2,655,-1,918.0,550.000000,0.827247,2,GRASS,-1,1520,0,114,99,2,2.209481,9.756119


In [13]:
subjects = np.unique(df_simu_g2.subject)
g2_stats = []
for subj in subjects:
    df_subj = df_simu_g2.query(f"subject == {subj}").copy()
    g2_stats.append(list(anal_perform(df_subj)))

In [14]:
print(np.array(g2_stats))
print(np.mean(np.array(g2_stats),axis=0))
print(sp.stats.sem(np.array(g2_stats),axis=0))

[[0.29166667 0.80833333 0.025      0.63414634]
 [0.33333333 0.75833333 0.00833333 0.86924035]
 [0.28333333 0.73333333 0.05833333 0.85375494]
 [0.35       0.75       0.00833333 0.88563727]
 [0.30833333 0.81666667 0.01666667 0.9296741 ]
 [0.35833333 0.80833333 0.00833333 0.72665699]
 [0.3        0.825      0.075      0.92222903]
 [0.20833333 0.78333333 0.03333333 0.90218156]
 [0.38333333 0.81666667 0.025      0.84837034]
 [0.30833333 0.75833333 0.01666667 0.95191884]
 [0.28333333 0.74166667 0.04166667 0.95020188]
 [0.21666667 0.825      0.05       0.87881286]
 [0.33333333 0.8        0.01666667 0.94463498]
 [0.23333333 0.75833333 0.025      0.3457189 ]
 [0.34166667 0.8        0.00833333 0.72154964]
 [0.28333333 0.73333333 0.025      0.64396285]
 [0.30833333 0.76666667 0.00833333 0.84428112]
 [0.25       0.78333333 0.05       0.76895944]
 [0.20833333 0.75       0.05833333 0.75744681]
 [0.3        0.79166667 0.03333333 0.55012853]
 [0.35833333 0.75833333 0.025      0.88448075]
 [0.24166667 

In [25]:
g2_stats = np.array(g2_stats)
g2_ds = sp.stats.norm.ppf(np.array(g2_stats[:,1])) - sp.stats.norm.ppf(np.array(g2_stats[:,2]))
print(np.mean(g2_ds))
print(sp.stats.sem(g2_ds))

2.6950624869881765
0.02795484230719647


### Item - CR (Group3)

In [15]:
tag = "Item-CR"
df_simu_g3 = simu_success(tag, params)
df_simu_g3

CMR2 Time: 135.88909697532654


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test,test_item1,test_item2,correct_ans,order,pair_idx,subject,group,corr_fin,omax_fin
0,0,0,609,-1,1.0,313.371643,0.607862,1,FROST,NaN,1,0,14,0,3,-1.000000,-1.000000
1,0,0,101,-1,0.0,380.427521,0.398559,1,BATTERY,NaN,0,-1,-1,0,3,-1.000000,-1.000000
2,0,0,927,-1,0.0,1126.790283,0.452851,1,MOVIE,NaN,1,1,13,0,3,-1.000000,-1.000000
3,0,0,1303,-1,1.0,795.575134,0.561278,1,SIDEWALK,NaN,1,0,31,0,3,-1.000000,-1.000000
4,0,0,945,-1,1.0,566.985840,0.578215,1,NEST,NaN,1,0,2,0,3,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47995,99,3,509,-1,992.0,260.000000,0.779693,2,DUSTPAN,NaN,128,1,149,99,3,1.614412,23.922749
47996,99,3,2,-1,992.0,270.000000,0.742875,2,ACID,NaN,970,1,163,99,3,1.465325,22.739393
47997,99,3,1463,-1,992.0,260.000000,0.762577,2,TICK,NaN,1255,0,171,99,3,1.471281,23.771116
47998,99,3,1615,-1,992.0,300.000000,0.653731,2,WINNER,NaN,667,1,157,99,3,1.352490,19.640743


In [16]:
subjects = np.unique(df_simu_g3.subject)
g3_stats = []
for subj in subjects:
    df_subj = df_simu_g3.query(f"subject == {subj}").copy()
    g3_stats.append(list(anal_perform(df_subj)))

In [17]:
print(np.array(g3_stats))
print(np.mean(np.array(g3_stats),axis=0))
print(sp.stats.sem(np.array(g3_stats),axis=0))

[[0.11875    0.5375     0.3625     0.65404837]
 [0.125      0.625      0.3        0.80935673]
 [0.175      0.55625    0.25625    0.20492454]
 [0.14375    0.60625    0.33125    0.31578947]
 [0.08125    0.53125    0.31875    0.63887542]
 [0.175      0.55625    0.28125    0.37569353]
 [0.1125     0.5125     0.30625    0.55599568]
 [0.0875     0.49375    0.3375     0.58030635]
 [0.1125     0.5125     0.225      0.44379774]
 [0.15       0.49375    0.3        0.47108604]
 [0.10625    0.6125     0.4125     0.92816219]
 [0.1375     0.5        0.29375    0.49135578]
 [0.06875    0.5625     0.3125     0.1406518 ]
 [0.1875     0.54375    0.325      0.73873023]
 [0.175      0.54375    0.16875    0.71259744]
 [0.1        0.61875    0.35625    0.44576659]
 [0.13125    0.56875    0.34375    0.54741816]
 [0.05625    0.53125    0.41875    0.47491639]
 [0.13125    0.5875     0.2625     0.6287695 ]
 [0.1875     0.5625     0.3375     0.4143827 ]
 [0.1        0.5375     0.33125    0.82791817]
 [0.1625     

In [26]:
g3_stats = np.array(g3_stats)
g3_ds = sp.stats.norm.ppf(np.array(g3_stats[:,1])) - sp.stats.norm.ppf(np.array(g3_stats[:,2]))
print(np.mean(g3_ds))
print(sp.stats.sem(g3_ds))

0.5951993149106226
0.01584890470737302


In [18]:
stats = []
stats.append(list(np.mean(np.array(g1_stats),axis=0)))
stats.append(list(np.mean(np.array(g2_stats),axis=0)))
stats.append(list(np.mean(np.array(g3_stats),axis=0)))
stats = np.array(stats)
stats

array([[0.3703    , 0.7815    , 0.1436    , 0.83094084],
       [0.30608333, 0.78091667, 0.03183333, 0.83100274],
       [0.12      , 0.546125  , 0.31775   , 0.52704099]])

In [19]:
ground_truth = np.array([[0.42, 0.72, 0.22, 0.81], [0.30, 0.80, 0.12, 0.71], [0.19, 0.67, 0.15, 0.57]])  # p_rc, hr, far, q
err = np.mean(np.power(stats - ground_truth,2))
err

0.007131214914108403